### Documentation pour l'explication du scraping du site FlashScore dans les archives de la Ligue 1

Dans un premier temps nous allons voir les importations qu'on aura besoin

In [4]:
from bs4 import BeautifulSoup #bibliothèque de scraping
import pandas as pd #Pour stocker les données
import time 
import re #permet d'utiliser re.compile
from selenium import webdriver #bibliothèque de scraping

Nous avons choisi quelles informations nous voulons scraper dans le site donc nous créons des tableaux pour ces données

In [5]:
domiciles=[]  #l'equipe à domicile
exterieurs=[] #l'equipe à l'exterieur
scoredoms=[]  #le score de l'equipe à domicile
scoreexts=[]  #le score de l'equipe à l'exterie
resultats=[]  #l'issue du match, le vainqueur ou Nul
saisons=[]    #la saison durant laquelle le match a eu lieu

Maintenant nous allons faire une boucle afin de récuperer les données sur plusieurs années donc differentes url.

    Pour acceder a tous les matchs nous devons appuyer sur le bouton Montrer plus ce qui correspond à la première partie de notre boucle. Pour ce faire il faut d'abord accépter les cookies.
    
    Notre boucle imbriqué nous permettera de récuperer ensuite tous les matchs durant la saison.
    
    Ensuite nous insérerons les données dans les tablaux créer au dessus pour enfin les transformer en une DataFrame.

In [6]:
for x in range(4,9) : #la boucle permet d'implanter dans l'url chaque saisons

    driver=webdriver.Chrome()  #permet d'ouvrir un navigateur 
    y=x+1
    driver.implicitly_wait(1)
    url= 'https://www.flashscore.fr/football/france/ligue-1-201'+str(x)+'-201'+str(y)+'/resultats/'
    driver.get(url)
    
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    time.sleep(5)
    
    #La boucle permet d'accepter les cookies
    Accept_button = soup.find('button', attrs={'id' :"onetrust-accept-btn-handler"})

    string = '$("#'+Accept_button.get('id')+'").click();'
    driver.execute_script(string)
    
    #Cette boucle permet d'afficher l'ensemble des matchs de la saison en cliquant sur Montrer plus
    more_matchs = driver.find_element_by_partial_link_text('Montrer plus de matchs')
    present = True
    while present == True :
        time.sleep(5)
        try :
            more_matchs.click()
            print("Succès du clique sur Montrer plus de matchs")
        except:
            present = False
            print("Affichage complet de la saison fini on passe au scraping")
        
    
    time.sleep(5)
    soup1 = BeautifulSoup(driver.page_source,'html.parser')
    
    #On cherche toutes les div contenant les informations de chaque match
    liste_matchs=soup1.find_all('div',class_=re.compile("event__match event__match--static"))
    nb_match = len(liste_matchs)
    print("Nombres de match trouvées : {}.".format(nb_match))
   
    #On scrap match par match les informations souhaités
    for i in liste_matchs :
        
        saison="201"+str(x)+"-201"+str(y)
        
        domicile=i.find('div',class_=re.compile("event__participant event__participant--home"))
        domicile=domicile.text
         
        exterieur=i.find('div',class_=re.compile("event__participant event__participant--away"))
        exterieur=exterieur.text
        
        score=i.find('div', class_="event__scores fontBold")
        scoredom=score.span.text
        scoreext=score.span.next_sibling.next_sibling.text
        
        scoredom=int(scoredom)
        scoreext=int(scoreext)
        
        #Permet de determiner l'issue du match 
        if scoredom > scoreext :
            resultat="VD"
        if scoredom < scoreext :
            resultat="VE"
        if scoredom == scoreext :
            resultat="N"
        
        #On ajoute les données de chaque match aux tableaux correspondants
        domiciles.append(domicile)
        exterieurs.append(exterieur)
        scoredoms.append(scoredom)
        scoreexts.append(scoreext)
        resultats.append(resultat)
        saisons.append(saison)
        
    driver.close()

Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 380.
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 380.
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 382.
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Affichage complet de la saison fini on passe au scraping
Nombres de match trouvées : 384.
Succès du clique sur Montrer plus de matchs
Succès du clique sur Montrer plus de matchs
Succès du clique sur Mon

On créer ensuite un DataFrame puis on l'ajoute dans un fichier csv afin d'analyser les données

In [7]:
matchs=pd.DataFrame({
    'saison':saisons,
    'equipe_dom':domiciles,
    'equipe_ext':exterieurs,
    'score_dom':scoredoms,
    'score_ext':scoreexts,
    'resultat':resultats})

matchs.to_csv(r'resultats_matchs.csv',sep=";",index=None)

On affiche les informations de la Dataframe pour verifier que les données sont bien rentrés

In [10]:
matchs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910 entries, 0 to 1909
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   saison      1910 non-null   object
 1   equipe_dom  1910 non-null   object
 2   equipe_ext  1910 non-null   object
 3   score_dom   1910 non-null   int64 
 4   score_ext   1910 non-null   int64 
 5   resultat    1910 non-null   object
dtypes: int64(2), object(4)
memory usage: 89.7+ KB


On affiche les premières données pour verifier que tout s'est bien deroulé

In [11]:
matchs.head()

,saison,equipe_dom,equipe_ext,score_dom,score_ext,resultat
0,2014-2015,Bordeaux,Montpellier,2,1,VD
1,2014-2015,Caen,Evian,3,2,VD
2,2014-2015,Lens,Nantes,1,0,VD
3,2014-2015,Lorient,Monaco,0,1,VE
4,2014-2015,Marseille,Bastia,3,0,VD
